In [ ]:
# Importing modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Initialising value
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# Loading directories
DIRECTORY = r'D:\Face-Mask-Detection-master\dataset'
CATEGORIES = ['with_mask', 'without_mask']
data = []
labels = []

In [ ]:
# Reading datasets and converting to array
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size= (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

In [ ]:
# Converting the labels into binary
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype='float32')
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state= 42)

In [ ]:
# Increasing dataset by augmentation
aug = ImageDataGenerator(
    rotation_range= 20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
#base model by MobileNet
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor= Input(shape=(224,224,3)))

#head of the model place on top of base model 
headModel = baseModel.output 
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='flattern')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

#freezing the baseModel so that weight will not updated 
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# Compiling and training
opt = Adam(lr = INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
             steps_per_epoch= len(trainX) // BS,
             validation_data=(testX, testY),
             validation_steps=len(testX) // BS,
             epochs = EPOCHS,)

In [ ]:
# Predicting the model
predIdxs = model.predict(testX, batch_size = BS)

# each testing image finding largest index label probability 
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testY.argmax(axis=1), predIdxs, target_names = lb.classes_))

model.save('mask_detector.model', save_format='h5')


In [ ]:
# Ploting accuracy
plt.figure(0)
plt.plot(H.history['acc'], label='training accuracy')
plt.plot(H.history['val_acc'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# Ploting loss
plt.plot(H.history['loss'], label= 'training loss')
plt.plot(H.history['val_loss'], label= 'val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Importing modules for testing
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import time 
import cv2
import imutils
import numpy as np

In [ ]:
# Function for detecting mask
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224), (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)
    
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2]
        
        if confidence > 0.5:
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype('int')
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX,startY, endX,endY))
            
    if len(faces):
        faces = np.array(faces, dtype='float32')
        preds = maskNet.predict(faces, batch_size= 32)
        
    return (locs,preds)

In [ ]:
# Loading facenet
prototxtPath = r'D:\Face-Mask-Detection-master\face_detector\deploy.prototxt'
weightsPath = r'D:\Face-Mask-Detection-master\face_detector\res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

#loading model 
maskNet = load_model('mask_detector.model')

In [ ]:
# Code for video capturing
cap = cv2.VideoCapture('Mask_test2.mp4')
while (cap.isOpened()):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (800, 600))
    
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    for (box,pred) in zip(locs,preds):
        
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred 
        
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask)* 100)
        
        cv2.putText(frame, label, (startX,startY-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX,startY), (endX,endY), color, 2)
        
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()